# Stem Separation

This was the initial implementaiton of our stem separator model that we used for our presentation demo. We've altered this code substantially since then, but we're including this in our final submission for reference.

## Imports & Setup

In [ ]:
%pip install nussl
%pip install git+https://github.com/source-separation/tutorial

In [ ]:
from common import data, utils

from tqdm import tqdm
from pathlib import Path
from collections import defaultdict

import IPython.display as display
from IPython.display import Audio

import nussl
from nussl.ml.networks.modules import AmplitudeToDB, BatchNorm, RecurrentStack, Embedding
from nussl.datasets import transforms as nussl_tfm
from ignite.engine import Events

import numpy as np
import pandas as pd
import torch
from torch import nn

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.mps.is_available() else 'cpu'

## Preparing the Data

In [ ]:
# Prepare MUSDB
data.prepare_musdb('~/.nussl/tutorial/')

In [ ]:
stft_params = nussl.STFTParams(window_length=512, hop_length=128, window_type='sqrt_hann')

tfm = nussl_tfm.Compose([
    nussl_tfm.SumSources([['bass', 'drums', 'other']]),
    nussl_tfm.MagnitudeSpectrumApproximation(),
    nussl_tfm.IndexSources('source_magnitudes', 1),
    nussl_tfm.ToSeparationModel(),
])

In [ ]:
train_folder = "~/.nussl/tutorial/train"
val_folder = "~/.nussl/tutorial/valid"

MAX_MIXTURES = int(1e8) # Set to some impossibly high number for on-the-fly mixing.

train_data = data.on_the_fly(stft_params, transform=tfm, fg_path=train_folder, num_mixtures=MAX_MIXTURES, coherent_prob=1.0)
train_dataloader = torch.utils.data.DataLoader(train_data, num_workers=1, batch_size=10)

val_data = data.on_the_fly(stft_params, transform=tfm, fg_path=val_folder, num_mixtures=10, coherent_prob=1.0)
val_dataloader = torch.utils.data.DataLoader(val_data, num_workers=1, batch_size=10)

In [ ]:
item = train_data[0]
for key in item:
    print(key, type(item[key]), item[key].shape if isinstance(item[key], torch.Tensor) else "")

In [ ]:
test_tfm = nussl_tfm.Compose([
    nussl_tfm.SumSources( # Only outputting vocals for our model
        groupings=[['drums', 'bass', 'other']],
        group_names=['accompaniment'],
    ),
    nussl_tfm.MagnitudeSpectrumApproximation(),
])

test_folder = "~/.nussl/tutorial/test"

test_data = data.on_the_fly(stft_params, transform=test_tfm, fg_path=test_folder, num_mixtures=100)

In [ ]:
item = test_data[0]
for key in item:
    print(key, type(item[key]), item[key].shape if isinstance(item[key], np.ndarray) else "")

## Defining the Model

In [ ]:
class StemSeparationModel(nn.Module):
    def __init__(self, num_features, num_audio_channels, hidden_size,
                 num_layers, bidirectional, dropout, num_sources, 
                activation='sigmoid'):
        super().__init__()
        
        self.verbose = False

        self.amplitude_to_db = AmplitudeToDB()
        self.input_normalization = BatchNorm(num_features)
        self.recurrent_stack = RecurrentStack(
            num_features * num_audio_channels, hidden_size, 
            num_layers, bool(bidirectional), dropout
        )
        hidden_size = hidden_size * (int(bidirectional) + 1)
        self.embedding = Embedding(num_features, hidden_size, 
                                   num_sources, activation, 
                                   num_audio_channels)
        
        self.set_up_config(num_features, num_audio_channels, hidden_size,
                 num_layers, bidirectional, dropout, num_sources, 
                activation)

    def set_up_config(self, num_features, num_audio_channels, hidden_size,
                 num_layers, bidirectional, dropout, num_sources, 
                activation='sigmoid'):
        modules = {
            'model': {
                'class': 'StemSeparationModel',
                'args': {
                    'num_features': num_features,
                    'num_audio_channels': num_audio_channels,
                    'hidden_size': hidden_size,
                    'num_layers': num_layers,
                    'bidirectional': bidirectional,
                    'dropout': dropout,
                    'num_sources': num_sources,
                    'activation': activation,
                }
            }
        }

        connections = [
            ['model', ['mix_magnitude']]
        ]

        for key in ['mask', 'vocal_estimate']:
            modules[key] = {'class': 'Alias'}
            connections.append([key, [f'model:{key}']])
        
        output = ['vocal_estimate', 'mask',]
        self.config = {
            'name': 'StemSeparationModel',
            'modules': modules,
            'connections': connections,
            'output': output,
        }
        self.metadata = {
            'config': self.config,
            'nussl_version': '0.0.1',
        }

    def log(self, s):
        if self.verbose:
            print(s)

    def forward(self, item):
        # Get magnitude of mixture signal
        mixture_magnitude = item['mix_magnitude']
        if mixture_magnitude.dim() == 3:
            mixture_magnitude = mixture_magnitude.unsqueeze(0)  # Add a batch dimension to the mixture magnitude if needed
        self.log(f"Shape of mixture_magnitude: {mixture_magnitude.shape}")

        # Convert to log amplitude
        mixture_log_amplitude = self.amplitude_to_db(mixture_magnitude)
        self.log(f"Shape after amplitude to db: {mixture_log_amplitude.shape}")
        
        # Normalize the data
        normalized = self.input_normalization(mixture_log_amplitude)
        self.log(f"Shape after normalization: {normalized.shape}")

        # Pass through LSTM
        output = self.recurrent_stack(normalized)
        self.log(f"Shape after LSTM: {output.shape}")

        # Generate mask
        mask = self.embedding(output)
        self.log(f"Shape of mask: {mask.shape}")
    
        # Apply mask to get estimates
        vocals_estimate = mixture_magnitude.unsqueeze(-1) * mask
        self.log(f"Shape of vocals estimate: {vocals_estimate.shape}")

        return {
            'mask': mask,
            'vocals_estimate': vocals_estimate,
        }

    def save(self, location, metadata=None, train_data=None, val_data=None, trainer=None):
        torch.save(self, location)
        return location
    
    def __repr__(self):
        output = super().__repr__()
        num_parameters = 0
        for p in self.parameters():
            if p.requires_grad:
                num_parameters += np.cumprod(p.size())[-1]
        output += '\nNumber of parameters: %d' % num_parameters
        return output

In [ ]:
num_features = stft_params.window_length // 2 + 1
num_audio_channels = 1
hidden_size = 50
num_layers = 2
bidirectional = True
dropout = 0.3
num_sources = 1
activation = 'sigmoid'

model = StemSeparationModel(
    num_features=num_features,
    num_audio_channels=num_audio_channels,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    num_sources=num_sources,
    activation=activation,
)

## Training the Model

In [ ]:
NUM_EPOCHS = 200
EPOCH_LENGTH = 10

model.verbose = False

utils.logger()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nussl.ml.train.loss.L1Loss()

def train_step(engine, batch):
    optimizer.zero_grad()
    output = model(batch) # forward pass
    loss = loss_fn(
        output['vocals_estimate'],
        batch['source_magnitudes']
    )
    
    loss.backward() # backwards + gradient step
    optimizer.step()
    
    loss_vals = {
        'L1Loss': loss.item(),
        'loss': loss.item()
    }

    progress_bar = engine.state.pbar
    progress_bar.set_description(f"Loss: {loss.item():.4f}")
    
    return loss_vals

def val_step(engine, batch):
    with torch.no_grad():
        output = model(batch) # forward pass
    loss = loss_fn(
        output['vocals_estimate'],
        batch['source_magnitudes']
    )    
    loss_vals = {
        'L1Loss': loss.item(), 
        'loss': loss.item()
    }
    return loss_vals

# Create the engines
trainer, validator = nussl.ml.train.create_train_and_validation_engines(
    train_step, val_step, device=DEVICE
)

@trainer.on(Events.EPOCH_STARTED)
def setup_pbar(engine):
    engine.state.pbar = tqdm(
        total=EPOCH_LENGTH,
        desc=f"Epoch {engine.state.epoch}/{NUM_EPOCHS}",
        leave=True,
    )

@trainer.on(Events.ITERATION_COMPLETED)
def update_pbar(engine):
    engine.state.pbar.update(1)

@trainer.on(Events.EPOCH_COMPLETED)
def close_pbar(engine):
    engine.state.pbar.close()

# We'll save the output relative to this notebook.
output_folder = Path('.').absolute()

# Adding handlers from nussl that print out details about model training
# run the validation step, and save the models.
nussl.ml.train.add_stdout_handler(trainer, validator)
nussl.ml.train.add_validate_and_checkpoint(output_folder, model, optimizer, train_data, trainer, val_dataloader, validator)

trainer.run(
    train_dataloader,
    epoch_length=EPOCH_LENGTH,
    max_epochs=NUM_EPOCHS,
)

## Deployment

In [ ]:
checkpoint_path = "checkpoints/200-epochs/best.model.pth"
model = torch.load(checkpoint_path, weights_only=False, map_location=DEVICE)

In [ ]:
def process_audio(item, model):    
    # Convert mixture signal to tensor if needed
    if isinstance(item['mix_magnitude'], np.ndarray):
        item['mix_magnitude'] = torch.from_numpy(item['mix_magnitude']).float().to(DEVICE)
    
    # Transpose for model input
    item["mix_magnitude"] = item["mix_magnitude"].transpose(0, 1)
    
    # Get model output (estimate of vocals source)
    with torch.no_grad():
        output = model(item)

    # Process the vocals estimate
    vocals_estimate = output['vocals_estimate']
    if vocals_estimate.dim() == 5:  # Remove extra dimensions
        vocals_estimate = vocals_estimate.squeeze(0).squeeze(-1).squeeze(-1)
    vocals_estimate = vocals_estimate.cpu().data.numpy()

    # Get the original mixture phase
    mix_stft = item['mix'].stft()
    mix_phase = np.angle(mix_stft)

    # Make sure vocals_estimate matches the original mixture phase
    # We want shape to be (freq_bins, time_frames)
    vocals_estimate = vocals_estimate.transpose()
    
    # Match shapes for combining magnitude and phase
    if vocals_estimate.shape[-1] == 1:
        vocals_estimate = vocals_estimate.squeeze(-1)
    if mix_phase.shape[-1] == 1:
        mix_phase = mix_phase.squeeze(-1)

    # Verify shapes of magnitude and phase match exactly
    assert vocals_estimate.shape == mix_phase.shape, f"Shape mismatch: vocals_estimate {vocals_estimate.shape} vs mix_phase {mix_phase.shape}"
    
    # Reconstruct complex STFT
    vocals_estimate_stft = vocals_estimate * np.exp(1j * mix_phase)
    
    # Create new audio signal with the same parameters as the input audio
    vocals_estimate_audio = nussl.AudioSignal(
        stft=vocals_estimate_stft,
        sample_rate=item['mix'].sample_rate,
        stft_params=item['mix'].stft_params,
    )

    # Perform inverse STFT
    vocals_estimate_audio.istft()
    
    # Ensure the output length matches the input exactly
    target_length = len(item['mix'].audio_data[0])
    current_length = len(vocals_estimate_audio.audio_data[0])
    
    # Pad the output audio to match the target length, if necessary
    if current_length != target_length:
        if current_length < target_length:
            pad_length = target_length - current_length
            vocals_estimate_audio.audio_data = np.pad(
                vocals_estimate_audio.audio_data, 
                ((0, 0), (0, pad_length)), 
                mode="constant",
            )
        else:
            vocals_estimate_audio.audio_data = vocals_estimate_audio.audio_data[:, :target_length]
    
    return vocals_estimate_audio

In [ ]:
SAVE_WAV_FILE = True
TEST_DATA_ITEM_INDEX = 4

item = test_data[TEST_DATA_ITEM_INDEX]
item['mix_magnitude'] = torch.from_numpy(item['mix_magnitude']).float().to(DEVICE)
vocals_estimate_audio = process_audio(item, model)
vocals_estimate_audio.embed_audio(display=False)

if SAVE_WAV_FILE:
    vocals_estimate_audio.write_audio_to_file(f"outputs/test-{TEST_DATA_ITEM_INDEX}-vocals-estimate.wav")

In [ ]:
print('true mixture')
display.display(Audio(data=item['mix'].audio_data, rate=item['mix'].sample_rate))

for stem_label in item['sources'].keys():
    print(f"true {stem_label}")
    audio_player = Audio(data=item['sources'][stem_label].audio_data, rate=item['sources'][stem_label].sample_rate)
    display.display(audio_player)

## Evaluation

In [ ]:
def get_scores(model, item):
    vocals_estimate_audio = process_audio(item, model)
    estimates = {
        "vocals": vocals_estimate_audio,
        "accompaniment": item["mix"] - vocals_estimate_audio,
    }

    sources = {
        "vocals": item["sources"]["vocals"],
        "accompaniment": item["sources"]["accompaniment"],
    }
    source_keys = list(sources.keys())

    estimates = [estimates[k] for k in source_keys]
    sources = [sources[k] for k in source_keys]

    evaluator = nussl.evaluation.BSSEvalScale(
        sources, estimates, source_labels=source_keys
    )
    scores = evaluator.evaluate()

    return scores

In [ ]:
METRICS = ["SI-SDR", "SI-SIR", "SI-SAR", "SD-SDR", "SNR", "SRR", "SI-SDRi", "SD-SDRi", "SNRi", "MIX-SI-SDR", "MIX-SD-SDR", "MIX-SNR"]
NUM_EVALUATION_ITEMS = len(test_data)

def evaluate_model(model, data, num_items):
    vocals_metrics, accompaniment_metrics = defaultdict(list), defaultdict(list)

    for i in range(num_items):
        item = data[i]
        scores = get_scores(model, item)

        for metric in METRICS:
            vocals_metrics[metric].extend(scores["vocals"][metric])
            accompaniment_metrics[metric].extend(scores["accompaniment"][metric])
    
    df = pd.DataFrame(columns=["Metric", "Vocals Mean", "Vocals Standard Deviation", "Accompaniment Mean", "Accompaniment Standard Deviation"])
    for metric in METRICS:
        metric_row = {
            "Metric": metric,
            "Vocals Mean": np.mean(vocals_metrics[metric]),
            "Vocals Standard Deviation": np.std(vocals_metrics[metric]),
            "Accompaniment Mean": np.mean(accompaniment_metrics[metric]),
            "Accompaniment Standard Deviation": np.std(accompaniment_metrics[metric]),
        }
        df = pd.concat([pd.DataFrame([metric_row], columns=df.columns), df], ignore_index=True)
    
    return df
        
eval_df = evaluate_model(model, test_data, NUM_EVALUATION_ITEMS)

In [ ]:
eval_df

In [ ]:
eval_df.to_csv("checkpoints/eval_df.csv", index=False)